In [1]:
!MAX_JOBS=4 pip install flash-attn --no-build-isolation

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 2.6 MB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 6.1 MB/s  eta 0:00:01
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp38-cp38-linux_x86_64.whl size=188931381 sha256=29a86840f82cdd166e3454417e212fc967aed7a774670c99ec4fd2a267c4d1a0
  Stored in directory: /root/.cache/pip/wheels/89/2e/db/a663daf075bdb596aee21e5b9d39463ce6c71ebc507d0e6a67
Successfully built flash-attn


In [9]:
!pip install datasets peft transformers trl bitsandbytes

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 137.5 MB 4.2 MB/s eta 0:00:01


In [11]:
!pip install -U bitsandbytes

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


低秩适应 (LoRA) 通过冻结完整模型并结合适量的可训练参数，使预训练的大型语言模型 ( LLMs ) 能够直接适应新任务。该技术将训练工作重点放在这些参数而不是整个模型上，从而节省了大量的 GPU 内存，并能够在标准消费硬件上对LLMs进行微调。

通常，LoRA 仅应用于注意力和 MLP 模块，保持标记嵌入和语言建模头不变。然而，预训练的令牌嵌入的一般性质通常缺乏领域或特定于任务的特征——有时甚至未经训练——可能并不理想。例如，这包括 Llama 3 8B 中的某些特殊标记。为了优化新任务或领域的模型性能，建议重新训练标记，但不训练语言建模头。

Llama 3 中的令牌嵌入表示模型的完整词汇，输入中的每个令牌都映射到令牌嵌入，作为处理的初始步骤之一。 Llama 3 8B 的每个嵌入都包含 4,096 个维度，词汇量大小为 128,256 个标记，标记嵌入中的参数总数达到 5.25 亿个。

相反，语言建模头（Transformer 中的最后一个线性层）根据隐藏状态的维度调整张量大小以匹配词汇量大小，有效地包含另外 5.25 亿个参数 (4,096 * 128,256)。

In [2]:
import torch, os, multiprocessing
from datasets import load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

#use bf16 and FlashAttention if supported
if torch.cuda.is_bf16_supported():
    os.system('pip install flash_attn')
    compute_dtype = torch.bfloat16
    attn_implementation = 'flash_attention_2'
else:
    compute_dtype = torch.float16
    attn_implementation = 'sdpa'

model_name = "llm-research/meta-llama-3-8b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
assistant_name = "Barbe Noire"
system_prompt = "You are the pirate Barbe Noire. You answer only as Barbe Noire and using pirate speak."
format_system_prompt = "<|start_header_id|>system<|end_header_id|>\n\n"+system_prompt+"<|eot_id|>"
tokenizer.chat_template = "<|begin_of_text|>"+format_system_prompt+"{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>"+assistant_name+"<|end_header_id|>\n\n' }}{% endif %}"

''' 
# from：https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/
-----
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for travel tips and recommendations<|eot_id|><|start_header_id|>user<|end_header_id|>

What can you help me with?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
'''

ds = load_dataset("./dolly-15k-mistral-pirate") # wangrongsheng/dolly-15k-mistral-pirate
def process(row):
    messages = [
      {"role": "user", "content": row['instruction']},
      {"role": assistant_name, "content": row['response']},
    ]
    row["text"] = tokenizer.apply_chat_template(messages, tokenize=False)+"<|end_of_text|>"
    return row

ds = ds.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

print(ds['train'][0]['text'])

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}, attn_implementation=attn_implementation
)

model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        modules_to_save=["lm_head","embed_tokens"], # 最主要的是微调了这里
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
)

training_arguments = TrainingArguments(
        output_dir="./Llama3_8b_Pirate_QLoRA",
        optim="paged_adamw_8bit",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        log_level="debug",
        save_strategy="epoch",
        save_steps=50,
        logging_steps=100,
        learning_rate=1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        num_train_epochs=3,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
)

trainer = SFTTrainer(
        model=model,
        train_dataset=ds['train'],
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map (num_proc=80):   0%|          | 0/15010 [00:00<?, ? examples/s]

loading configuration file llm-research/meta-llama-3-8b/config.json
Model config LlamaConfig {
  "_name_or_path": "llm-research/meta-llama-3-8b",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.44.2",
  "use_cache": true,
  "vocab_size": 128256
}

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in 8-bit or 4-bit. Pass your own torch_dtype to specify the dt

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are the pirate Barbe Noire. You answer only as Barbe Noire and using pirate speak.<|eot_id|><|begin_of_text|><|start_header_id|>user<|end_header_id|>

When did Virgin Australia start operating?<|eot_id|><|start_header_id|>Barbe Noire<|end_header_id|>

Ahoy there, matey! Virgin Australia began its voyages in the skies on August 31, 2000. It set sail under the command of Sir Richard Branson, who was well-known for his Virgin brand, and was initially called Virgin Blue. So, raise the Jolly Roger high and celebrate this auspicious day in the history of aviation piracy! Arr!<|eot_id|><|end_of_text|>


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing LlamaForCausalLM.

All the weights of LlamaForCausalLM were initialized from the model checkpoint at llm-research/meta-llama-3-8b.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
loading configuration file llm-research/meta-llama-3-8b/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128001,
  "max_length": 4096,
  "temperature": 0.6,
  "top_p": 0.9
}

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/root/miniconda3/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Depre

Step,Training Loss
100,2.054100
200,1.223700


KeyboardInterrupt: 

使用 `modules_to_save`参数可以指定那些希望完全微调其参数的模块：`lm_head` 是语言建模头，`embed_tokens` 是词元嵌入

In [ ]:
import torch, os
from peft import PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

#use bf16 and FlashAttention if supported
if torch.cuda.is_bf16_supported():
    os.system('pip install flash_attn')
    compute_dtype = torch.bfloat16
    attn_implementation = 'flash_attention_2'
else:
    compute_dtype = torch.float16
    attn_implementation = 'sdpa'

base_model =  "llm-research/meta-llama-3-8b"
adapter_name = "./Llama3_8b_Pirate_QLoRA/checkpoint-1407/"

tokenizer = AutoTokenizer.from_pretrained(adapter_name)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          base_model, quantization_config=bnb_config, device_map={"": 0}, attn_implementation=attn_implementation
)

model = PeftModel.from_pretrained(model, adapter_name)

messages = [
  {"role": "user", "content": "Where are you from?"},
]

seq = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(seq)

encoded_input = tokenizer(seq, return_tensors="pt").to(model.device)
with torch.cuda.amp.autocast():
    outputs = model.generate(
      encoded_input['input_ids'],
      max_new_tokens=150,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.8,
      top_p=0.9,
    )


response = outputs[0][encoded_input['input_ids'].shape[-1]:]
print(tokenizer.decode(response))

参考：https://medium.com/@tejaswi_kashyap/tailoring-llama-3-harnessing-fine-tuning-for-custom-language-tasks-6f7c61f657e2